# RAG 기반 고객 상담 챗봇 만들기

이 노트북에서는 LangChain과 ChromaDB를 사용하여 **RAG(Retrieval-Augmented Generation)** 기반의 고객 상담 챗봇을 구축하는 방법을 배웁니다.

## 학습 목표

| 주제 | 설명 |
|------|------|
| 문서 로딩 | 마크다운 파일을 LangChain Document로 로드 |
| 청킹 | 문서를 적절한 크기로 분할하는 전략 |
| 임베딩 | 텍스트를 벡터로 변환하여 저장 |
| 검색 | 질문과 유사한 문서 검색 |
| RAG 체인 | 검색된 문서를 활용한 LLM 답변 생성 |
| 웹 UI | Gradio로 대화형 인터페이스 구축 |

---

## 1. 프로젝트 구조

```
example/
├── app.py                    # Gradio 웹 애플리케이션
├── src/
│   ├── embed_documents.py    # 문서 임베딩 모듈
│   └── rag_chain.py          # RAG 체인 모듈
├── knowledge_base/           # 지식베이스 문서
│   ├── products/             # 제품 정보
│   ├── policies/             # 정책 문서 (배송, 교환/환불, 보증)
│   └── faq/                  # FAQ
└── chroma_db/                # 벡터 DB (자동 생성)
```

### 데이터 흐름

```
1. 문서 준비 (knowledge_base/)
       ↓
2. 문서 로드 및 청킹 (embed_documents.py)
       ↓
3. 임베딩 & 벡터 DB 저장 (chroma_db/)
       ↓
4. 사용자 질문 → 유사 문서 검색 (rag_chain.py)
       ↓
5. 검색 결과 + 질문 → LLM 답변 생성
       ↓
6. Gradio UI로 응답 표시 (app.py)
```

## 2. 환경 설정

In [ ]:
# 필요한 패키지 (이미 설치되어 있다면 생략)
# !pip install langchain langchain-openai langchain-chroma langchain-huggingface langchain-community gradio python-dotenv

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ OpenAI API 키 로드됨: {api_key[:8]}...")
else:
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

In [ ]:
# 경로 설정
EXAMPLE_DIR = Path("./example")
KNOWLEDGE_BASE_DIR = EXAMPLE_DIR / "knowledge_base"
CHROMA_DB_DIR = EXAMPLE_DIR / "chroma_db"

print(f"예제 폴더: {EXAMPLE_DIR.absolute()}")
print(f"지식베이스: {KNOWLEDGE_BASE_DIR.absolute()}")

## 3. 지식베이스 문서 살펴보기

우리가 준비한 지식베이스에는 가상의 쇼핑몰 "TechMall"의 정보가 들어있습니다.

In [ ]:
# 지식베이스 구조 확인
print("📁 지식베이스 구조:\n")

for category_dir in sorted(KNOWLEDGE_BASE_DIR.iterdir()):
    if category_dir.is_dir():
        files = list(category_dir.glob("*.md"))
        print(f"📂 {category_dir.name}/")
        for f in files:
            print(f"   └─ {f.name}")

In [ ]:
# 샘플 문서 내용 확인
sample_file = KNOWLEDGE_BASE_DIR / "products" / "smartwatch_pro.md"

print(f"📄 {sample_file.name} 내용:\n")
print(sample_file.read_text(encoding="utf-8")[:1000])

## 4. 문서 로딩 (Document Loading)

LangChain의 `DirectoryLoader`를 사용하여 마크다운 파일들을 로드합니다.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

def load_all_documents(base_path: Path) -> list:
    """
    지식베이스의 모든 마크다운 문서를 로드합니다.
    각 문서에 카테고리 메타데이터를 추가합니다.
    """
    documents = []
    
    for category_folder in base_path.iterdir():
        if not category_folder.is_dir():
            continue
        
        category = category_folder.name
        
        # DirectoryLoader로 해당 폴더의 .md 파일 로드
        loader = DirectoryLoader(
            str(category_folder),
            glob="**/*.md",
            loader_cls=TextLoader,
            loader_kwargs={"encoding": "utf-8"}
        )
        
        docs = loader.load()
        
        # 메타데이터에 카테고리 추가
        for doc in docs:
            doc.metadata["category"] = category
            doc.metadata["filename"] = Path(doc.metadata["source"]).name
            documents.append(doc)
    
    return documents

In [ ]:
# 문서 로드
documents = load_all_documents(KNOWLEDGE_BASE_DIR)

print(f"총 {len(documents)}개 문서 로드됨\n")

# 로드된 문서 정보 출력
for doc in documents:
    content_preview = doc.page_content[:50].replace("\n", " ") + "..."
    print(f"📄 [{doc.metadata['category']}] {doc.metadata['filename']}")
    print(f"   내용: {content_preview}")
    print(f"   길이: {len(doc.page_content):,}자\n")

## 5. 청킹 (Chunking)

문서가 너무 길면 검색 효율이 떨어집니다. 적절한 크기로 분할해야 합니다.

### 청킹 전략

| 파라미터 | 설명 | 권장값 |
|---------|------|-------|
| chunk_size | 청크의 최대 문자 수 | 300-1000 |
| chunk_overlap | 청크 간 겹치는 문자 수 | chunk_size의 10-20% |
| separators | 분할 기준 문자열 | 헤더, 단락 순서로 |

### RecursiveCharacterTextSplitter

가장 많이 사용되는 텍스트 분할기입니다. 지정된 separators 순서대로 분할을 시도합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n## ", "\n### ", "\n\n", "\n", " ", ""]
)

# 문서 분할
chunks = text_splitter.split_documents(documents)

print(f"원본 문서 수: {len(documents)}")
print(f"청크 수: {len(chunks)}")
print(f"평균 청크 크기: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f}자")

In [ ]:
# 청크 샘플 확인
print("=" * 60)
print("청크 샘플 (처음 3개)")
print("=" * 60)

for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- 청크 {i+1} ---")
    print(f"카테고리: {chunk.metadata['category']}")
    print(f"파일: {chunk.metadata['filename']}")
    print(f"길이: {len(chunk.page_content)}자")
    print(f"내용:\n{chunk.page_content[:300]}...")

## 6. 임베딩 & 벡터 저장 (Embedding & Vector Store)

청크를 벡터로 변환하여 ChromaDB에 저장합니다.

### 임베딩 모델 선택

| 모델 | 차원 | 특징 |
|------|-----|------|
| all-MiniLM-L6-v2 | 384 | 무료, 빠름, 로컬 실행 |
| text-embedding-3-small | 1536 | OpenAI, 유료, 고성능 |
| text-embedding-3-large | 3072 | OpenAI, 유료, 최고 성능 |

이번 예제에서는 무료로 로컬에서 실행 가능한 HuggingFace 모델을 사용합니다.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import shutil

# 임베딩 모델 초기화
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

print(f"임베딩 모델 로드 중: {EMBEDDING_MODEL}")
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
print("✅ 임베딩 모델 로드 완료")

In [ ]:
# 임베딩 테스트
test_text = "스마트워치 배터리 수명"
test_embedding = embedding_model.embed_query(test_text)

print(f"테스트 텍스트: '{test_text}'")
print(f"임베딩 차원: {len(test_embedding)}")
print(f"임베딩 샘플: {test_embedding[:5]}")

In [ ]:
# 기존 DB 삭제 (재생성을 위해)
if CHROMA_DB_DIR.exists():
    shutil.rmtree(CHROMA_DB_DIR)
    print("기존 벡터 DB 삭제됨")

# 벡터 스토어 생성
print("\n벡터 DB 생성 중...")

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=str(CHROMA_DB_DIR)
)

# 저장 정보 확인
collection = vector_store._collection
print(f"\n✅ 벡터 DB 생성 완료")
print(f"   저장된 벡터 수: {collection.count():,}개")
print(f"   저장 경로: {CHROMA_DB_DIR}")

## 7. 유사 문서 검색 (Retrieval)

벡터 DB에서 질문과 유사한 문서를 검색합니다.

In [ ]:
# Retriever 생성
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 문서 반환
)

# 검색 테스트
test_query = "배송은 얼마나 걸리나요?"
retrieved_docs = retriever.invoke(test_query)

print(f"질문: {test_query}")
print(f"\n검색된 문서 수: {len(retrieved_docs)}\n")
print("=" * 60)

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[{i}] {doc.metadata['category']} / {doc.metadata['filename']}")
    print(f"{doc.page_content[:200]}...")

In [ ]:
# 다른 질문으로 테스트
queries = [
    "환불하려면 어떻게 해야 하나요?",
    "스마트워치 방수 되나요?",
    "포인트는 어떻게 사용하나요?"
]

for query in queries:
    docs = retriever.invoke(query)
    print(f"\n🔍 '{query}'")
    print(f"   → [{docs[0].metadata['category']}] {docs[0].page_content[:80]}...")

## 8. RAG 체인 구현

검색된 문서를 바탕으로 LLM이 답변을 생성하도록 합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

# 시스템 프롬프트 템플릿
SYSTEM_PROMPT = """당신은 TechMall 쇼핑몰의 친절한 고객 상담원입니다.
고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.

## 답변 지침
- 아래 참고 자료를 기반으로 답변하세요.
- 참고 자료에 없는 내용은 "확인 후 안내드리겠습니다"라고 말씀해주세요.
- 친절하고 전문적인 톤을 유지하세요.

## 참고 자료
{context}
"""

print("LLM 준비 완료")

In [ ]:
def ask_with_rag(question: str) -> str:
    """
    RAG 파이프라인으로 질문에 답변합니다.
    """
    # 1. 관련 문서 검색
    docs = retriever.invoke(question)
    
    # 2. 컨텍스트 구성
    context_parts = []
    for doc in docs:
        category = doc.metadata.get("category", "")
        context_parts.append(f"[{category}]\n{doc.page_content}")
    context = "\n\n---\n\n".join(context_parts)
    
    # 3. 프롬프트 구성
    system_message = SYSTEM_PROMPT.format(context=context)
    
    # 4. LLM 호출
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=question)
    ]
    
    response = llm.invoke(messages)
    return response.content

In [ ]:
# RAG 테스트
test_questions = [
    "SmartWatch Pro X1 가격이 얼마인가요?",
    "환불은 어떻게 신청하나요?",
    "보증 기간을 연장할 수 있나요?"
]

for q in test_questions:
    print(f"\n{'='*60}")
    print(f"Q: {q}")
    print(f"{'='*60}")
    answer = ask_with_rag(q)
    print(f"\nA: {answer}")

## 9. RAG vs 일반 LLM 비교

RAG를 사용하지 않으면 어떻게 될까요?

In [ ]:
def ask_without_rag(question: str) -> str:
    """RAG 없이 LLM에 직접 질문"""
    messages = [
        SystemMessage(content="당신은 TechMall 쇼핑몰의 고객 상담원입니다."),
        HumanMessage(content=question)
    ]
    response = llm.invoke(messages)
    return response.content

# 비교
comparison_question = "SmartWatch Pro X1의 배터리 수명은 얼마인가요?"

print("질문:", comparison_question)
print("\n" + "="*60)
print("📚 RAG 사용 (지식베이스 참조)")
print("="*60)
print(ask_with_rag(comparison_question))

print("\n" + "="*60)
print("❌ RAG 미사용 (LLM 지식만)")
print("="*60)
print(ask_without_rag(comparison_question))

## 10. Gradio UI로 챗봇 실행

이제 완성된 RAG 챗봇을 Gradio UI로 실행해봅시다.

터미널에서 다음 명령을 실행하세요:

```bash
cd basic/day6/example
python app.py
```

또는 아래 셀을 실행하여 간단한 챗 인터페이스를 띄울 수 있습니다.

In [ ]:
import gradio as gr

def chat_fn(message, history):
    """Gradio용 채팅 함수"""
    return ask_with_rag(message)

# 간단한 채팅 UI
demo = gr.ChatInterface(
    fn=chat_fn,
    title="TechMall 고객 상담 챗봇",
    description="제품, 배송, 교환/환불, 보증 등에 대해 물어보세요!",
    examples=[
        "SmartWatch Pro X1 가격이 얼마인가요?",
        "환불은 어떻게 신청하나요?",
        "배송은 얼마나 걸리나요?"
    ]
)

demo.launch()

## 11. 요약

### RAG 파이프라인 구성요소

| 단계 | 모듈 | 역할 |
|------|------|------|
| 1. 문서 로딩 | DirectoryLoader | 파일 시스템에서 문서 읽기 |
| 2. 청킹 | RecursiveCharacterTextSplitter | 문서를 적절한 크기로 분할 |
| 3. 임베딩 | HuggingFaceEmbeddings | 텍스트를 벡터로 변환 |
| 4. 저장 | ChromaDB | 벡터 저장 및 검색 |
| 5. 검색 | Retriever | 유사 문서 검색 |
| 6. 생성 | ChatOpenAI | 컨텍스트 기반 답변 생성 |
| 7. UI | Gradio | 대화형 웹 인터페이스 |

### 핵심 포인트

1. **청킹 전략이 중요**: chunk_size와 overlap을 문서 특성에 맞게 조정
2. **메타데이터 활용**: 카테고리, 출처 등을 저장하여 필터링/디버깅에 활용
3. **프롬프트 엔지니어링**: 시스템 프롬프트로 LLM의 동작 방식 제어
4. **RAG의 장점**: 최신 정보 반영, 환각 감소, 출처 추적 가능

### 다음 단계

- 하이브리드 검색 (BM25 + 벡터)
- Re-ranking으로 검색 품질 향상
- 스트리밍 응답
- 대화 이력 관리